In [1]:
library(SBC);
library(cmdstanr);
library(tidyverse);
options(mc.cores = parallel::detectCores());
library(future);
plan(multisession);

options(SBC.min_chunk_size = 5);

cache_dir <- "./SBC_cache"
if(!dir.exists(cache_dir)) {
    dir.create(cache_dir)
}

model_Poisson_v2 <- cmdstanr::cmdstan_model("../models/Poisson_model_v2.stan")

This is cmdstanr version 0.7.1

- CmdStanR documentation and vignettes: mc-stan.org/cmdstanr

- CmdStan path: /Users/igor.michels/.cmdstan/cmdstan-2.34.1

- CmdStan version: 2.34.1

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message in readLines(stan_file):
“linha final incompleta encontrada em '../models/Poisson_model_v2.stan'”


In [2]:
data_generator_single_Poisson_v2 <- function(n_clubs, n_players_per_clubs){
    n_seasons <- 1
    clubs <- 1:n_clubs
    force <- abs(rnorm(length(clubs) * n_players_per_clubs))
    df <- data.frame(Club = clubs)
    data <- merge(df, df, by = NULL) %>% filter(Club.x != Club.y)
    data <- data %>% slice(rep(row_number(), n_seasons))
    home_goals <- list()
    away_goals <- list()
    home_players <- array(NA, dim = c(nrow(data), 11))
    away_players <- array(NA, dim = c(nrow(data), 11))
    players <- seq(1, n_players_per_clubs)
    for (i in 1:nrow(data)) {
        home_team <- data[i, 1]
        away_team <- data[i, 2]
        home_players_game <- sample(players)[1:11]
        away_players_game <- sample(players)[1:11]
        home_force <- sum(force[(home_team - 1) *  n_players_per_clubs + home_players_game])
        away_force <- sum(force[(away_team - 1) *  n_players_per_clubs + away_players_game])
        home_goals <- append(home_goals, rpois(1, home_force / away_force))
        away_goals <- append(away_goals, rpois(1, away_force / home_force))
        home_players[i,] <- home_players_game
        away_players[i,] <- home_players_game
    }

    data$home.goals <- home_goals
    data$away.goals <- away_goals
    names(data) <- c("home_name", "away_name", "home_goals", "away_goals")

    list(
        variables = list(
            skills = force
        ),
        generated = list(
            n_games = nrow(data),
            n_teams = n_clubs,
            n_players_per_team = n_players_per_clubs,
            home_team = data$home_name,
            away_team = data$away_name,
            home_score = data$home_goals,
            away_score = data$away_goals,
            home_players = home_players,
            away_players = away_players
        )
    )
}

In [3]:
run_model <- function(n_players_per_clubs){
    set.seed(0)
    n_sims <- 250
    data_generator_Poisson_v2 <- SBC_generator_function(data_generator_single_Poisson_v2, n_clubs = 20,
                                                        n_players_per_clubs = n_players_per_clubs)

    dataset_Poisson_v2 <- generate_datasets(data_generator_Poisson_v2, n_sims)
    
    # max_rank will be iter_sampling * chains / 10 - 1
    backend_Poisson_v2 <- SBC_backend_cmdstan_sample(model_Poisson_v2, iter_warmup = 2000,
                                                     iter_sampling = 2500, chains = 4)

    results_Poisson_v2 <- compute_SBC(dataset_Poisson_v2, backend_Poisson_v2,
                                      keep_fits = FALSE,
                                      cache_mode = "results",
                                      cache_location = file.path(cache_dir,
                                                                 sprintf("results_Poisson_v2_%02d_players_per_club",
                                                                         n_players_per_clubs)))
    
    write.csv(results_Poisson_v2$stats,
              file = sprintf("SBC_cache/results_Poisson_v2_%02d_players_per_club_stats.csv",
                             n_players_per_clubs))
    
    results_Poisson_v2
}

In [4]:
for (n_players_per_clubs in 11:20) {
    results <- run_model(n_players_per_clubs)
}

Results loaded from cache file 'results_Poisson_v2_11_players_per_club'

Results loaded from cache file 'results_Poisson_v2_12_players_per_club'

Results loaded from cache file 'results_Poisson_v2_13_players_per_club'

Results loaded from cache file 'results_Poisson_v2_14_players_per_club'

Results loaded from cache file 'results_Poisson_v2_15_players_per_club'

Results loaded from cache file 'results_Poisson_v2_16_players_per_club'

Results loaded from cache file 'results_Poisson_v2_17_players_per_club'

Results loaded from cache file 'results_Poisson_v2_18_players_per_club'

Results loaded from cache file 'results_Poisson_v2_19_players_per_club'

Results loaded from cache file 'results_Poisson_v2_20_players_per_club'

